In [1]:
import pandas as pd
import numpy as np
import random
import string 
import tensorflow as tf
import cv2

In [2]:
!mkdir images

mkdir: cannot create directory ‘images’: File exists


# Generate CAPTCHA images

## Install Python's captcha library

In [3]:
!pip install captcha

In [4]:
from captcha.image import ImageCaptcha

## CAPTCHA images 

 the function generates images  include 5 characters :
 - letters and numbers
 -  symbols and punctuations (optional) 

input:
- ratio  
    - letters : numbers : symbols
- split percentage 

output:
- Dictionary { keys -> train + validate + test , values -> list [ image path' , tag ] }


In [5]:
def CAPTCHAsgenerating (datasize: int , imgsize : tuple , ratio :tuple ,tr_s :float ):
  v_s = float((1-tr_s)/2)
  data_dict = {'tr':[],'val':[],'ts':[]}
  captcha =[]
  image = ImageCaptcha(width = imgsize[0] , height = imgsize[1] )
  punctuation =string.punctuation
  list(punctuation).remove("/")
  for i in range(datasize):
    letters = [random.choice(string.ascii_letters) for i in range(ratio[0]) ]
    numbers = [random.choice(string.digits       ) for i in range(ratio[1]) ]
    symbols = [random.choice(punctuation         ) for i in range(ratio[2]) if not '/']
    captcha_text = [str(elem) for elem in letters+numbers+symbols ] 
    random.shuffle( captcha_text) 
    captcha_text = ''.join(captcha_text)
    path = '/content/images/'+captcha_text+'.png'
    captcha.append ([path,captcha_text])
    data = image.generate(captcha_text) 
    image.write(captcha_text, path )
    tr, val , ts = np.split(captcha, [int(len(captcha)*tr_s), int(len(captcha)*(1-v_s)) ])
    data_dict['tr'].extend(tr)
    data_dict['val'].extend(val)
    data_dict['ts'].extend(ts)
    
  return data_dict

In [6]:
data_dict= CAPTCHAsgenerating(datasize= 20  , imgsize =(280 , 90) , ratio = (2,2,1) ,tr_s =0.8 )

In [ ]:
print (data_dict['val'])

## Data Pipeline

In [ ]:
class pipeline(tf.keras.utils.Sequence):
  def __init__(self,input_x,labels,resize_shape,batch_size,shuffle=True):
    self.x = input_x  # pipeline input 
    self.y = labels   # pipeline output
    self.resize_shape = resize_shape
    # The pipeline needs to take ''' batch size ( 8 examples , 16 examples, 32 example, 48 example)
    # and shuffle paremeter [ true - false ] to shuffle or not shuffle the data
    self.batch_size = batch_size  
    self.shuffle    = shuffle 
    self.on_epoch_end()

  def __len__(self):
    # This function determines the number of batches
    return int(np.floor(len(self.y) / self.batch_size))

  def __getitem__(self, index):
    # Get the current batch 
    indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
    X, y = self.__get_batch(indexes)
    return X,y

  def on_epoch_end(self):
    self.indexes = np.arange(len(self.x))
    if self.shuffle == True:
      np.random.shuffle(self.indexes)

  def __read_image(self,path):
    ''' a function that reads the image and do resizing changes the color convention'''
    img=cv2.imread(path)
    dim = self.resize_shape[0:2]
    #for interpolation in enlarging: cv2.INTER_LANCZOS4 : reducing: cv2.INTER_AREA  
    interpolation = cv2.INTER_AREA if((self.resize_shape[0]*self.resize_shape[1])<=(img.shape[0]*img.shape[1])) else cv2.INTER_LANCZOS4
    if (img.shape[2]!=3):
      img = cv2.cvtColor(img,cv2.COLOR_GRAY2RGB)
    return(cv2.resize(img,dim,interpolation =interpolation))/255.0
    

  def __get_label(self,label_string):
    ''' a function that converts string to the class number '''
    return label_string



  def __get_batch(self, list_IDs_temp):
    '''
    Does the following three main things:
    1- Create two arrays for input and output with correct shapes
    '''
    X = np.empty((self.batch_size, *self.resize_shape),dtype=np.float32)
    y = np.empty((self.batch_size, 3))

    # Generate data
    for i, ID in enumerate(list_IDs_temp):
      # Get expression
      X[i,:,:] = self.__read_image(self.x[ID])

      # store label
      y[i,]    = self.__get_label(self.y[ID])

    return X, y

### Build pipeline

In [ ]:
train_x= np.array( np.array( data_dict['tr' ] ) [:,0] )
train_y= np.array( np.array( data_dict['tr' ] ) [:,1] )
test_x = np.array( np.array( data_dict['ts' ] ) [:,0] )
test_y = np.array( np.array( data_dict['ts' ] ) [:,1] )
val_x  = np.array( np.array( data_dict['val'] ) [:,0] )
val_y  = np.array( np.array( data_dict['val'] ) [:,1] )

In [ ]:
resize_shape= (32,32,3)
batch_size = 16
epochs     = 40

train_generator = pipeline(input_x=train_x,labels = train_y
                           ,resize_shape = resize_shape ,batch_size=batch_size)

validation_generator = pipeline(input_x = val_x , labels = val_y
                           ,resize_shape = resize_shape ,batch_size=batch_size)

test_generator = pipeline(input_x = test_x , labels = test_y
                           ,resize_shape = resize_shape ,batch_size=batch_size)